<a href="https://colab.research.google.com/github/JTassinari10/456FinalCNN/blob/main/Nathan_Taylor_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Nathan Taylor CECS 456 Final Project
Natural Images CNN Classification model

#Importing libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
import sklearn
import sklearn.model_selection
import cv2
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from google.colab import drive
import os
np.random.seed(42)

#Importing dataset
Choose archive.zip downloaded from kaggle

In [ ]:
from google.colab import files

if "natural_images" not in os.listdir("./"):
  files.upload()
  os.system("unzip \*archive.zip")
  os.system("rm archive.zip")

In [ ]:
for directory_name, _, file_names in os.walk("/natural_images"):
  for file_name in file_names:
    (os.path.join(directory_name, file_name))

labels = os.listdir("./natural_images")
mapping = {}
for i in range(len(labels)):
  mapping[labels[i]] = i

def resize_image(img):
  return cv2.resize(image, (180, 180), interpolation=cv2.INTER_AREA)

X_dataset_full = []
y_dataset_full = []
for label in labels:
  path = "./natural_images/{0}/".format(label)
  image_paths = os.listdir(path)
  for image_path in image_paths:
    image = cv2.imread(path + image_path)
    image = resize_image(image)
    X_dataset_full.append(np.array(image))
    y_dataset_full.append(mapping[label])

In [ ]:
#Converting dataset to np arrays
X_dataset_full = np.array(X_dataset_full)
y_dataset_full = np.array(y_dataset_full)
#Standardizing data
X_dataset_full = X_dataset_full.astype("float32")/255

In [ ]:
#Shuffling data
r = np.arange(X_dataset_full.shape[0])
np.random.shuffle(r)
X = X_dataset_full[r]
Y = y_dataset_full[r]

In [ ]:
#Splitting dataset by giving 33% to testing
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, Y, test_size=0.33
)

In [ ]:
cnn = tf.keras.models.Sequential() #Initializing the CNN
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=5, activation="relu", padding = "same", input_shape=[180, 180, 3])) #Convolution 1
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) #Pooling 1
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding = "same")) #Convolution 2
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding = "same")) #Convolution 3
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) #Pooling 2
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu", padding = "same")) #Convolution 4
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu", padding = "same")) #Convolution 5
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) #Pooling 3
tf.keras.layers.Dropout(0.5) #Dropout
cnn.add(tf.keras.layers.Flatten()) #Flattening
cnn.add(tf.keras.layers.Dense(units=128, activation="relu"))#Full Connection
tf.keras.layers.Dropout(0.5) #Dropout
cnn.add(tf.keras.layers.Dense(units= 8, activation= "softmax"))#Output layer

In [ ]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 180, 180, 64)      4864      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 128)       73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 90, 90, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 128)      0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 45, 45, 256)       2

In [ ]:
cnn.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [ ]:
#Freeing up some RAM
X_dataset_full = 0
y_dataset_full = 0

In [ ]:
history = cnn.fit(X_train, y_train, batch_size=100, epochs=8, validation_split=0.1)

Epoch 1/8
42/42 [==============================] - 37s 584ms/step - loss: 1.7226 - accuracy: 0.4213 - val_loss: 0.9003 - val_accuracy: 0.6890
Epoch 2/8
42/42 [==============================] - 18s 436ms/step - loss: 0.5829 - accuracy: 0.7802 - val_loss: 0.5189 - val_accuracy: 0.7948
Epoch 3/8
42/42 [==============================] - 18s 440ms/step - loss: 0.3466 - accuracy: 0.8726 - val_loss: 0.4213 - val_accuracy: 0.8229
Epoch 4/8
42/42 [==============================] - 19s 444ms/step - loss: 0.2504 - accuracy: 0.9017 - val_loss: 0.3585 - val_accuracy: 0.8553
Epoch 5/8
42/42 [==============================] - 19s 449ms/step - loss: 0.1798 - accuracy: 0.9344 - val_loss: 0.3826 - val_accuracy: 0.8575
Epoch 6/8
42/42 [==============================] - 19s 453ms/step - loss: 0.1106 - accuracy: 0.9553 - val_loss: 0.4591 - val_accuracy: 0.8510
Epoch 7/8
42/42 [==============================] - 19s 454ms/step - loss: 0.1372 - accuracy: 0.9531 - val_loss: 0.6223 - val_accuracy: 0.8359
Epoch 

#Evaluation

In [ ]:
score = cnn.evaluate(X_test, y_test)
print('Total loss on Testing Set:', score[0])
print('Accuracy of Testing Set:', score[1])

72/72 [==============================] - 5s 59ms/step - loss: 0.4997 - accuracy: 0.8718
Total loss on Testing Set: 0.4997088313102722
Accuracy of Testing Set: 0.8717610836029053


In [ ]:
X_new = X_test[:20]
y_pred = np.argmax(cnn.predict(X_new), axis=-1)
print(y_pred)
print(y_test[:20])

[2 2 2 2 6 0 6 7 0 4 4 1 1 4 3 4 1 7 1 1]
[2 2 5 5 6 0 6 7 5 4 4 1 1 4 3 4 1 7 1 1]
